In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tarfile
import urllib.request
import os

MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz '
MODEL_FILE = MODEL_URL.split('/')[-1]
MODEL_DIR = MODEL_FILE.split('.')[0]

if not os.path.exists(MODEL_FILE):
    urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)
    tar = tarfile.open(MODEL_FILE)
    tar.extractall()
    tar.close()

PATH_TO_SAVED_MODEL = os.path.join(MODEL_DIR, "saved_model")
print('Loading model...', end='')
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')
def load_label_map(label_path):
  labels = {}
  with open(label_path, 'r') as file:
    item = {}
    for line in file:
      if line.strip().startwith('id:'):
        item['id'] = int(line.strip().split(' ')[1])
      if line.strip().startwith('display_name:'):
        item['display_name'] = line.strip().split('"')[1]
      if 'id' in item and 'display_name' in item:
        labels[item['id']]  = item['display_name']
        item = {}
  return labels


labels = load_label_map('/content/mscoco_label_map.pbtxt')


def load_image_into_tensor(image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]
    return input_tensor

def draw_detections(image, detections, threshold=0.65):
    scores = detections['detection_scores'][0].numpy()
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)

    for i in range(len(scores)):
        if scores[i] >= threshold:
            box = boxes[i] * [image.shape[0], image.shape[1], image.shape[0], image.shape[1]]
            y_min, x_min, y_max, x_max = box.astype(int)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            class_name = labels.get(classes[i], 'N/A')
            label = f'{class_name}: {scores[i]:.2f}'
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cap = cv2.VideoCapture('/content/video1.mp4')
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    input_tensor = load_image_into_tensor(frame)
    detections = detect_fn(input_tensor)

    draw_detections(frame, detections)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


Loading model...Done!
